# CTE

A common table expression is a named temporary result set that exists only within the execution scope of a single SQL statement e.g.,SELECT, INSERT, UPDATE, or DELETE.

The basic syntax of a CTE is as follows:

```mysql
WITH cte_name (column_list) AS (
    # a query
)
# a query referencing to cte_name
```

(column_list) is optional. The number of columns in column_list should be the same as that in the query, but they don't have the same names. The column names of the cte is those in column_list. If (column_list) is not given, the column names of the query inside the cte are used.


```mysql
WITH ctl AS (
    ...
)
SELECT ... 
FROM tbl INNER JOIN ctl USING (x);
```

Multiple CTEs can be used in the same query.

```mysql
WITH ctl1 AS (
    # a query
),
ctl2 AS (
    # a query referencing to ctl1
) 
SELECT ...
```

## Recursive CTE

A CTE can be recursive:

```mysql
WITH RECURSIVE cte_name AS (
    # initial query called the anchor member
    UNION ALL # or UNION DISTINCT
    # a query referencing to cte_name, called the recursive member
)
# a query referencing to cte_name
```

The recursive member 

* should have a condition stopping the recursion,

* cannot contain aggregate functions, GROUP BY, ORDER BY, and LIMIT, 

* cannot contain DISTINCT if UNION ALL is used,

* can reference the cte only once and in its FROM clause.



The following example uses a recursive CTE generating a row sequentially and unioning all.

```mysql
WITH RECURSIVE cte_count (n) 
AS (
      SELECT 1
      UNION ALL
      SELECT n + 1 
      FROM cte_count 
      WHERE n < 3              # A termination condition is necessary.
    )
SELECT n 
FROM cte_count;

+------+
| n    |
+------+
|    1 |
|    2 |
|    3 |
+------+
```


Assume that we have a table named tree. id and pid (parent id) are among the columns of the table. Each row in the table corresponds to a node in a tree structure. The pid of the root node is NULL. We want to create a query showing id, pid, and depth in ascending order of depth, where depth is the depth of a node (1 for the root node, 2 for the children nodes of the root node, and so on).

Plan:

1. Create the root node: (id, pid=NULL, depth=1)

2. Assume the current cte contains all records with depth=k. To construct the next cte containing all records with depth=k+1, we join the current cte with tree on tree.pid = cte.id

```mysql
WITH RECURSIVE cte AS (
  SELECT id, pid, 1 depth FROM tree WHERE pid IS NULL
  UNION ALL
  SELECT t.id, t.pid, depth+1 FROM tree AS t INNER JOIN cte ON cte.id = t.pid)
SELECT id, pid, depth
FROM cte
ORDER BY depth;
```

An example from https://www.mysqltutorial.org/mysql-recursive-cte/ is as follows:

```mysql
WITH RECURSIVE employee_paths AS (
    SELECT employeeNumber, # id
           reportsTo managerNumber, # pid
           officeCode,   # will be used when we join the cte with another table
           1 lvl    # depth
    FROM employees
    WHERE reportsTo IS NULL
    UNION ALL
    SELECT e.employeeNumber, e.reportsTo, e.officeCode, lvl+1
    FROM employees e INNER JOIN employee_paths ep ON ep.employeeNumber = e.reportsTo)
SELECT employeeNumber, managerNumber, lvl, city
FROM employee_paths ep INNER JOIN offices o USING (officeCode)
ORDER BY lvl, city;
```

# Functions

Use DECLARE to define local variables. If we define something using SET in a function or procedure, it is not a local variable. 

```sql
CREATE FUNCTION city_pop(city_name CHAR(35), country_code CHAR(3))
RETURNS INT READS SQL DATA
BEGIN
  DECLARE pop INT;
  DECLARE CONTINUE HANDLER FOR NOT FOUND SET pop = 0;
  SELECT Population INTO pop FROM world.city 
  WHERE CountryCode = country_code AND Name = city_name;
  RETURN pop;
END

SELECT city_pop('Chicago', 'USA');
```

## Window functions

* CUME_DIST() calculates the cumulative distribution of a value within a group of values.


* LAG() and LEAD() accesse a row at a specific physical offset before/after the current row, respectively. The first argument is the element to be returned, the second argument is the offset (1 by default), and the third argument is the default value to return if there is no row at the requested offset.

```sql
SELECT a, b, LAG(b) OVER (PARTITION BY a ORDER BY c) FROM tbl;
```

* FIRST_VALUE(), LAST_VALUE()


* ROW_NUMBER(), RANK(), DENSE_RAND()


* NTILE(): The following is used to divide rows into four groups based on x values. If there are 14 rows, then the group number of the first four rows is 1, the group number of the next four rows is 2, the group number of the next three rows is 3, and the group number of the last three rows is 4.

```sql
NTILE(4) OVER (ORDER BY x DESC)
```

If value = 10,20,20,30,50,50 and we compute ROW_NUMBER(), RANK(), and DENSE_RANK() OVER (ORDER BY value), 

then row_number = 1,2,3,4,5,6, rank = 1,2,2,4,5,5, and dense_rank = 1,2,2,3,4,4.


```sql
SELECT x, ROW_NUMBER() OVER (ORDER BY x) w_x FROM tbl;

SELECT x, y, ROW_NUMBER() OVER (ORDER BY y DESC) w_y FROM tbl;

SELECT x, y, z, ROW_NUMBER() OVER(PARTITION BY z ORDER BY y DESC) w_z FROM tbl;


SELECT x, y, z, SUM(z) OVER(PARTITION BY z 
                            ORDER BY y
                            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
FROM tbl;

-- UNBOUNDED PRECEDING
-- CURRENT ROW
-- UNBOUNDED FOLLOWING
```

## Inline table-valued functions

```sql
CREATE FUNCTION func(@x AS INT) RETURNS TABLE
AS
RETURN
  SELECT ...
  FROM tbl
  WHERE id = @x;
  
SELECT ...
FROM func(2) AS t;

SELECT ...
FROM func(3) AS tbl1 INNER JOIN tbl2 ON ...;
```

# Prepared statements

If MySQL server executes a prepared statement, it does not need to fully parse the query. It will simply use the value assigned in the placeholder.

```sql
SET @stmt = 'SELECT * FROM tbl';
PREPARE qry FROM @stmt;
EXECUTE qry;
DEALLOCATE PREPARE qry;
```
In the above example, assume that the table name tbl does not exists. Then we have the error ERROR 1146 (42S02): Table 'currentdb.tbl' doesn't exist. Note that we have the error in the prepare stage, not in the execute stage. If there is an error in the prepare stage, we cannot do the execute and deallocate prepare statements.


```sql
SET @stmt = 'SELECT x, y FROM tbl WHERE x = ?';           # ? is a placeholder.
PREPARE qry FROM @stmt;
SET @x_val = 5; EXECUTE qry USING @x_val;
SET @x_val = 7; EXECUTE qry USING @x_val;
DEALLOCATE PREPARE qry;
```

An example from https://www.mysqltutorial.org/

```sql
SET @schema = 'classicmodels';
SET @pattern = 'o%';
SELECT CONCAT('DROP TABLE ', GROUP_CONCAT(CONCAT(@schema, '.', TABLE_NAME)), ';')
INTO @droplike
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = @schema AND TABLE_NAME LIKE @pattern;

SELECT @droplike;
# DROP TABLE classicmodels.offices,classicmodels.orderdetails,classicmodels.orders;

PREPARE stmt FROM @droplike;
EXECUTE stmt;
DEALLOCATE PREPARE stmt;
```

# Procedure

## MySQL

Example:

```sql
DELIMITER //

CREATE PROCEDURE max_city_pop(IN country_code CHAR(3), OUT city_name CHAR(35), OUT pop INT)
BEGIN
  SELECT IFNULL(MAX(Population),0) INTO pop FROM world.city 
  WHERE CountryCode = country_code;
  
  SELECT IF(MAX(Name) IS NULL, 'Unknown', Name) INTO city_name FROM world.city 
  WHERE CountryCode = country_code AND Population = pop;
END

DELIMITER ;

CALL max_city_pop('KOR', @cityname, @maxpop);
SELECT @cityname, @maxpop;
```

Example from https://www.mysqltutorial.org/

```sql
DELIMITER //

CREATE PROCEDURE check_table_exists(IN table_name VARCHAR(100), OUT table_exists TINYINT(1))
BEGIN
    DECLARE CONTINUE HANDLER FOR 1146 SET @err = 1;
    SET @err = 0;
    SET @table_name = table_name;
    SET @query = CONCAT('SELECT 1 FROM ', @table_name, ';');
    PREPARE stmt FROM @query;
    SELECT IF(@err = 1, 0, 1) INTO table_exists;
    IF(@err=0) THEN
        DEALLOCATE PREPARE stmt;
    END IF;
END //

CALL check_table_exists('my_tbl', @table_exists);
SELECT @table_exists;
```

Note that in the above example we don't use an EXECUTE statement. It is because we will have an error in the PREPARE stage when table_name does not exist.

## SQL Server

```sql
DROP PROC IF EXISTS proc_name;
GO

CREATE PROC proc_name
  @x AS NVARCHAR(50)
AS
SELECT ...
FROM ...
WHERE col = @x;
GO

EXEC proc_name @x=10;
```

# SIGNAL

In a stored function, procedure, or trigger,

```sql
IF ... THEN
    SIGNAL SQLSTATE 'state_value' SET MYSQL_ERRNO = err_no, MESSAGE_TEXT = 'msg';
END IF;
```